In [2]:
from src.luxai_s3.wrappers import LuxAIS3GymEnv
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import math
from collections import deque
from torch.distributions import Categorical
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
torch.nan_to_num((torch.zeros(1)+1e-18) * (-torch.inf))

tensor([-3.4028e+38])

In [4]:
ac = torch.from_numpy(np.array([[0,12,56],[5,13,45],[4,13,4],[5,2,3]]))
a = ac[:,0]!=5

ac[a,2] = 0
ac[a,1] = 0
print(ac)

tensor([[ 0,  0,  0],
        [ 5, 13, 45],
        [ 4,  0,  0],
        [ 5,  2,  3]])


In [5]:
a = torch.zeros(5,2,6)
a[:,0] = 1
print(a)
a = torch.sum(a.flatten(0,1),axis=1)
print(a)
print(a.view(5,2))

tensor([[[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]]])
tensor([6., 0., 6., 0., 6., 0., 6., 0., 6., 0.])
tensor([[6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.]])


In [6]:
indices = torch.randperm(100).type(torch.IntTensor)
x = torch.rand((100))
x[indices[:100]]

tensor([0.9222, 0.2657, 0.4916, 0.1879, 0.2267, 0.2679, 0.7085, 0.4933, 0.7835,
        0.7093, 0.7055, 0.6212, 0.3844, 0.6577, 0.4315, 0.2095, 0.7125, 0.0408,
        0.6713, 0.7041, 0.1687, 0.7226, 0.2692, 0.9140, 0.8823, 0.6316, 0.9785,
        0.5398, 0.7482, 0.6746, 0.8606, 0.4367, 0.8786, 0.2093, 0.0598, 0.3685,
        0.9630, 0.8879, 0.7511, 0.3586, 0.8945, 0.4366, 0.0141, 0.7525, 0.9023,
        0.2374, 0.3581, 0.2860, 0.3066, 0.2734, 0.2051, 0.4178, 0.2372, 0.5948,
        0.9992, 0.7167, 0.7581, 0.1484, 0.4648, 0.3979, 0.6950, 0.2053, 0.4335,
        0.9759, 0.3784, 0.2863, 0.7060, 0.2716, 0.9769, 0.7289, 0.6581, 0.0144,
        0.6840, 0.5636, 0.9439, 0.2429, 0.0073, 0.9550, 0.1586, 0.5050, 0.2378,
        0.0962, 0.2508, 0.1529, 0.1721, 0.7328, 0.4797, 0.5213, 0.5009, 0.3228,
        0.7631, 0.7661, 0.9558, 0.2313, 0.4361, 0.5867, 0.1394, 0.1689, 0.2163,
        0.9850])

In [7]:
Categorical(probs=torch.Tensor([[0.3,0.3,0.3]])).sample()

tensor([0])

In [8]:
env = LuxAIS3GymEnv(numpy_output=True)
obs,_ = env.reset()
print(env.env_params)


EnvParams(max_steps_in_match=100, map_type=1, map_width=24, map_height=24, num_teams=2, match_count_per_episode=5, max_units=16, init_unit_energy=100, min_unit_energy=0, max_unit_energy=400, unit_move_cost=4, spawn_rate=3, unit_sap_cost=46, unit_sap_range=4, unit_sap_dropoff_factor=0.5, unit_energy_void_factor=0.125, max_energy_nodes=6, max_energy_per_tile=20, min_energy_per_tile=-20, max_relic_nodes=6, relic_config_size=5, fog_of_war=True, unit_sensor_range=2, nebula_tile_vision_reduction=0, nebula_tile_energy_reduction=0, nebula_tile_drift_speed=0.02500000037252903, energy_node_drift_speed=0.029999999329447746, energy_node_drift_magnitude=5)


In [51]:
env = LuxAIS3GymEnv(numpy_output=True)
obs,_ = env.reset()

for i in tqdm(range(50)) :
    action = dict(
                player_0=np.random.randint(0,4,size=(env.env_params.max_units, 3)),
                player_1=np.random.randint(0,4,size=(env.env_params.max_units, 3))
            )
    obs, reward, terminated, truncated, info = env.step(action)

print(obs['player_0']['map_features'])


100%|██████████| 50/50 [00:03<00:00, 13.97it/s]

{'energy': array([[ 0, -4, -2,  4,  8,  6, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [-2, -4,  1,  6,  7,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [-3, -2,  3,  7,  5, -1, -5, -2, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [-3,  0,  5,  7,  2, -3, -4,  2, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [-3,  1,  6,  5,  0, -3, -2,  4,  7, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [-2,  2,  5,  4,  0, -3,  0,  5,  6, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  2,  5,  3, -1, -2,  1,  5,  5, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0,  2,  4,  2,  0, -1,  2,  4,  3, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1,  3,  2,  1,  1,  2,  2,  2, -1, -1, -1, -1, -1, -1, -1,
        -1, -

In [54]:
torch.ones(16,1)*(torch.arange(0,16)).view(16,1)

tensor([[ 0.],
        [ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.],
        [11.],
        [12.],
        [13.],
        [14.],
        [15.]])

In [52]:
print(obs['player_0'])

{'units': {'position': array([[[ 7,  5],
        [ 7,  4],
        [10,  4],
        [ 6,  4],
        [ 4,  5],
        [ 5,  0],
        [ 2,  3],
        [ 4,  4],
        [ 6,  6],
        [ 3,  3],
        [ 1,  0],
        [ 3,  2],
        [ 3,  2],
        [ 2,  2],
        [ 0,  1],
        [ 2,  2]],

       [[-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1]]], dtype=int16), 'energy': array([[  0,   0,   8,  55,   0,   4, 129,  71,  60,  51,  30,  70,  94,
         73,  70,  79],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1]], dtype=int16)}, 'units_mask': array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [False, False

In [113]:
torch.where(torch.tensor([[True,False,True],[False,True,True]]))

(tensor([0, 0, 1, 1]), tensor([0, 2, 1, 2]))

In [80]:
def obs_to_state(obs:dict) -> torch.Tensor:
    list_state_features = []

    state_maps = torch.zeros(3,24,24)
    
    state_maps[0] = torch.from_numpy(obs['map_features']['energy'].astype(float)) #map_energy
    state_maps[1] = torch.from_numpy(obs['sensor_mask'].astype(float)) #sensor_mask
    state_maps[2] = torch.from_numpy(obs['map_features']['tile_type'].astype(float)) #map_tile_type

    #Units
    list_state_features.append(torch.from_numpy(obs['units']['position'].astype(float)).flatten()/24) #position
    list_state_features.append(torch.from_numpy(obs['units']['energy'].astype(float)).flatten()/400) #energy
    list_state_features.append(torch.from_numpy(obs['units_mask'].astype(float)).flatten()) #unit_mask

    list_state_features.append(torch.from_numpy(obs['relic_nodes'].astype(float)).flatten()) #relic_nodes
    list_state_features.append(torch.from_numpy(obs['relic_nodes_mask'].astype(float)).flatten()) #relic_nodes_mask

    #Game
    list_state_features.append(torch.from_numpy(obs['team_points'].astype(float)).flatten()/3000) #team_points
    list_state_features.append(torch.from_numpy(obs['team_wins'].astype(float)).flatten()/5) #team_wins

    list_state_features.append(torch.from_numpy(obs['steps'].astype(float)).flatten()/100) #steps
    list_state_features.append(torch.from_numpy(obs['match_steps'].astype(float)).flatten()/5) #match_steps
    
    state = {'features':torch.cat(list_state_features),'maps':state_maps}

    return state

class Policy(nn.Module) :

    def __init__(self,state,n_action,env_param,player) :

        super(Policy,self).__init__()

        if player == 'player_0' :
            self.player_id = 0
        else :
            self.player_id = 1

        self.n_action = n_action
        self.n_units = env_param.max_units
        self.sap_range = env_param.unit_sap_range
        self.unit_move_cost = env_param.unit_move_cost
        self.unit_sap_cost = env_param.unit_sap_cost

        self.network_size = [2048,1024,512,64]
        self.cnn_channels = [3,9,27]
        self.cnn_kernels = [3,5,9]
        self.cnn_strides = [1,1,1]
    
        self.n_input_maps = state['maps'].size(0)

        self.activation = nn.ReLU()
        self.max_pooling = nn.MaxPool2d(kernel_size=2)

        self.cnn_inputs = nn.Conv2d(self.n_input_maps,
                                    self.cnn_channels[0],
                                    kernel_size=self.cnn_kernels[0],
                                    padding=self.cnn_kernels[0]-1,
                                    stride=self.cnn_strides[0])
        
        self.cnn_hidden = [nn.Conv2d(self.cnn_channels[i],
                                     self.cnn_channels[i+1],
                                     kernel_size=self.cnn_kernels[i+1],
                                     padding=self.cnn_kernels[i+1]-1,
                                    stride=self.cnn_strides[i+1]) for i in range(len(self.cnn_channels)-1)]
        
        with torch.no_grad() :
            x = self.cnn_inputs(state["maps"])
            x = self.max_pooling(x)
            for layer in self.cnn_hidden :
                x = layer(x)
                x = self.max_pooling(x)
            self.n_input_features = x.flatten().size(0) + state['features'].size(0)
        
        self.inputs = nn.Linear(self.n_input_features,self.network_size[0],dtype=torch.double)
        self.hidden = [nn.Linear(self.network_size[i],self.network_size[i+1],dtype=torch.double) for i in range(len(self.network_size)-1)]

        self.actor_action = nn.Linear(self.network_size[-1],self.n_action*self.n_units,dtype=torch.double)
        self.actor_dx = nn.Linear(self.network_size[-1],(self.sap_range*2+1)*self.n_units,dtype=torch.double)
        self.actor_dy = nn.Linear(self.network_size[-1],(self.sap_range*2+1)*self.n_units,dtype=torch.double)

        self.critic = nn.Linear(self.network_size[-1],1,dtype=torch.double)

    def training_forward(self,x,action,mask_action,mask_dx,mask_dy) :

        x = self.activation(self.cnn_inputs(x['maps']))
        x = self.max_pooling(x)
        for layer in self.cnn_hidden :
            x = self.activation(layer(x))
            x = self.max_pooling(x)

        x = torch.cat((x.flatten(start_dim=1),x['features']),dim=0)

        x = self.activation(self.inputs(x))
        for layer in self.hidden :
            x = self.activation(layer(x))
        x = self.activation(self.outputs(x))

        actor_action = self.actor_action(x).view(-1,self.n_units,self.n_action) + torch.nan_to_num(mask_action*(-torch.inf))
        actor_dx = self.actor_dx(x).view(-1,self.n_units,self.sap_range*2+1) + torch.nan_to_num(mask_dx*(-torch.inf))
        actor_dy = self.actor_dy(x).view(-1,self.n_units,self.sap_range*2+1) + torch.nan_to_num(mask_dy*(-torch.inf))

        actor_action = F.log_softmax(actor_action,dim=-1)
        actor_dx = F.log_softmax(actor_dx,dim=-1)
        actor_dy = F.log_softmax(actor_dy,dim=-1)

        value = self.critic(x)

        # Computing log probabilities for the actions

        batch_size = actor_action.size(0)
        n_units = actor_action.size(1)

        step_indices = torch.arange(batch_size).unsqueeze(1).expand(-1, n_units)
        unit_indices = torch.arange(n_units).unsqueeze(0).expand(batch_size, -1) 

        log_prob = torch.sum(actor_action[step_indices,unit_indices, action[:,:, 0]],axis=1)
        log_prob += torch.sum(actor_dx[step_indices,unit_indices, action[:,:, 1]+self.sap_range],axis=1)
        log_prob += torch.sum(actor_dy[step_indices,unit_indices, action[:,:, 2]+self.sap_range],axis=1)

        return value,log_prob.view(-1,1)

    def forward(self,x,obs) :

        x = self.activation(self.cnn_inputs(x['maps']))
        x = self.max_pooling(x)
        for layer in self.cnn_hidden :
            x = self.activation(layer(x))
            x = self.max_pooling(x)

        print(x.size(),x['features'].size())

        x = torch.cat((x.flatten(start_dim=0),x['features']),dim=0)

        x = self.activation(self.inputs(x))
        for layer in self.hidden :
            x = self.activation(layer(x))
        x = self.activation(self.outputs(x))

        state = {}
        state['energy'] = torch.from_numpy(obs['units']['energy'][self.player_id].astype(int)).view(self.n_units)
        state['units'] = torch.from_numpy(obs['units']['position'][self.player_id].astype(int)).view(self.n_units,2)
        state['map'] = torch.from_numpy(obs['map_features']['tile_type'].astype(int)).view(24,24)
        

        energy_mask = state['energy'] < self.unit_move_cost
        sap_mask = state['energy'] < self.unit_sap_cost

        mask_action = torch.zeros(self.n_units,self.n_action)
        mask_dx = torch.zeros(self.n_units,self.sap_range*2+1)
        mask_dy = torch.zeros(self.n_units,self.sap_range*2+1)

        mask_action[torch.where(energy_mask)[0],1:] += 1
        mask_action[torch.where(sap_mask)[0],-1] += 1

        directions = torch.tensor([[0,-1],[1,0],[0,1],[-1,0]]).view(4,2)
        target_tiles = state['units'].unsqueeze(1).expand(self.n_units, 4, 2) + directions
        clamp_target_tiles = torch.clamp(target_tiles,0,23).view(self.n_units*4,2)
        target_tiles_type = state['map'][clamp_target_tiles[:,0],clamp_target_tiles[:,1]].view(self.n_units,4)

        correct_move_direction = (((target_tiles >= 0) & (target_tiles <= 23)).all(dim=-1)) & (target_tiles_type != 2)
        forbidden_move = 1 - correct_move_direction.int() 
        mask_action[:,1:-1] += forbidden_move
        
        
        actor_action = self.actor_action(x).view(self.n_units,self.n_action) + torch.nan_to_num(mask_action*(-torch.inf))
        actor_action = F.log_softmax(actor_action,dim=-1)
        action_choice = Categorical(logits=actor_action).sample()

        sap_mask =  sap_mask | (action_choice !=5)

        mask_dx[torch.where(sap_mask)[0]] = 1
        mask_dy[torch.where(sap_mask)[0]] = 1
        mask_dx[:,self.sap_range] = 0
        mask_dy[:,self.sap_range] = 0

        actor_dx = self.actor_dx(x).view(self.n_units,self.sap_range*2+1) + torch.nan_to_num(mask_dx*(-torch.inf))
        actor_dy = self.actor_dy(x).view(self.n_units,self.sap_range*2+1) + torch.nan_to_num(mask_dy*(-torch.inf))
        actor_dx = F.log_softmax(actor_dx,dim=-1)
        actor_dy = F.log_softmax(actor_dy,dim=-1)

        # Sampling action based on the policy
        action = torch.zeros(self.n_units, 3, dtype=torch.int)

        action[:, 0] = action_choice
        action[:, 1] = Categorical(logits=actor_dx).sample() - self.sap_range
        action[:, 2] = Categorical(logits=actor_dy).sample() - self.sap_range

        value = self.critic(x)

        return action, value, mask_action, mask_dx, mask_dy 
    

state = obs_to_state(obs['player_0'])

model = Policy(state,6,env.env_params,'player0')
model(state,obs)


IndexError: too many indices for tensor of dimension 3

tensor(-0.3929, dtype=torch.float64) tensor(0.9731, dtype=torch.float64)
